# Startup Stuff
Importing libraries, defining functions & variables, opening ports, and camera settings.

In [ ]:
from zaber.serial import BinarySerial, BinaryDevice, BinaryCommand, BinaryReply, UnexpectedReplyError, TimeoutError
# for zaber library binary protocol manual go here: https://www.zaber.com/wiki/Manuals/Binary_Protocol_Manual
import time
import numpy as np        #importing numpy package to create random matrix
import usb.core
import usb.util
import sys
import binascii
import png
import time
import serial
import numpy as np
import matplotlib.pyplot as plt
from array import *
import time as t
import os
import sys

# This is the section where we define our settings:


# Open a serial port. You may need to edit this section for your particular
# hardware and OS setup.
# Variables: BinarySerial(port,baud,timeout,inter_char_timeout)
# Ports: Linux="/dev/ttyUSB0"; Baud rate - all Zaber Binary devices use 9600
# Timeout: number of seconds to wait for a reply (fractions accepted)
# inter_char_timeout: number of seconds to wait between bytes in a reply
port = BinarySerial("COM3", baud = 9600, timeout = 120, inter_char_timeout=180)         # Windows

# Get a handle for device #1 on the serial chain. This assumes you have a
# device already in Binary 9,600 baud mode at address 1 on your port.
device = BinaryDevice(port,1)# Device number 1
device2 = BinaryDevice(port,2)

# this is where the images will be saved
#foldername = "Mightex Camera"      # this is a new folder that the program will create and save to                 
directory = "C:/Users/NU-ACCESS/Documents/MightexDataFiles" # this is where the folder will be saved
 
    
numframes = 1   # this is the number of frames for each acquisition


#This section is defining our functions

#             Error Handling for the Stages

#                   Danielle: the constant printing was starting to bother me when I started moving to further distances, so I commented it.
def can_ignore(reply):
    if reply.command_number == 8: # Move tracking is enabled
        #print("Move tracking from device " + str(reply.device_number) + ": position = " + str(reply.data))
        return True
    elif reply.command_number == 10: # Move tracking is enabled
        #print("Manual move tracking from device " + str(reply.device_number) + ": position = " + str(reply.data))
        return True
    elif reply.command_number == 11: # Move tracking is enabled
        #print("Manual move was triggered on device " + str(reply.device_number))
        return True
    
    return False

def safe_move(device, position):
    retry = False
    try:
        reply = device.move_abs(position)
        if can_ignore(reply):
            retry = True
        else:
            return check_command_succeeded(reply)
    except UnexpectedReplyError:
        retry = True
    except TimeoutError:
        print("Communication timeout!")
        return False

    while retry:
        try:            
            reply = port.read()
        except TimeoutError:
            print("Communication timeout!")
            return False

        if can_ignore(reply):
            retry = True
        else:
            retry = False
            if not check_command_succeeded(reply):
                return False

    return True


def check_command_succeeded(reply):
    """
    Return true if command succeeded, print reason and return false if command
    rejected

    param reply: BinaryReply

    return: boolean
    """
    if reply.command_number == 255: # 255 is the binary error response code.
        print ("Danger! Command rejected by device " + str(reply.device_number) + ". Error code: " + str(reply.data))
        return False
    
    # else command was accepted
    return True


# Define Camera class to control Mightex SCE-BG04-U CMOS Camera
# Define all functions for Camera class
# This is where you should change camera settings, such as exposure time, gain, resolution, etc.
class Camera:
	def __init__(self,res=(752,480),exposure_time=7.5,gain=8,fps=10):   #fps=10   exp time:0.25
		
		self.dev = usb.core.find(idVendor=0x04B4, idProduct=0x0228)
		
		# If no device is found
		if self.dev is None:
			raise ValueError('Device not found')
		else:
			pass
			
		# Sets camera to default configuration
		self.dev.set_configuration()
		
		# make sure we can read and write
		# not sure why this is necessary, but without this commands timeout
		r = self.dev.write(0x01,[0x21])
		r = self.dev.write(0x01,[0x21])
		r = self.dev.write(0x01,[0x21])
		r = self.dev.read(0x81,0x2E)

		self.get_device_info()
		
		self.set_mode(0x00)
		self.res = res
		self.exposure_time = exposure_time
		self.gain = gain
		self.fps = fps
		self.set_gain(self.gain)
		self.set_resolution(self.res)
		self.set_exposure_time(self.exposure_time)
		self.set_fps(self.fps)
		self.set_sensor_Hblanking()
		self.set_main_clock_freq()
		time.sleep(2)
		
	def get_device_info(self):
		self.dev.write(0x01,[0x21,0x01,0x00])
		info = self.dev.read(0x81,0x2E)
		#print("Info: " + str(info))
		
	def get_firmware_version(self):
		self.dev.write(0x01,[0x01])
		res = self.dev.read(0x81,0x05)
		return res.tolist()
		
	def set_mode(self,mode):
		self.dev.write(0x01, [0x30,0x01,mode])
			
	def set_resolution(self,res):
		xres = self.int2hexlist(res[0])
		yres = self.int2hexlist(res[1])
		result = self.dev.write(0x01, [0x60, 0x07, xres[0], xres[1], yres[0], yres[1], 0x00])
		self.res = res
		return result
			
	def set_gain(self,gain):
		if getattr(gain,'__iter__',False):
			if not len(gain) == 3:
				raise ValueError("Gain tuple must consist of exactly three values")
			res = self.dev.write(0x01,[0x62,0x03,gain[0],gain[1],gain[2]])
		else:
			res = self.dev.write(0x01,[0x62,0x03,gain,gain,gain])
			
		self.gain = gain
		
	def set_exposure_time(self,time):
		time_mult = int(time/0.05)
		time_mult = self.int2hexlist(time_mult)
		#print("Exposure time set to: " + str(time_mult) + "*0.05ms")
		self.dev.write(0x01, [0x63, 0x02, time_mult[0],time_mult[1]])
		self.exposure_time = float(time)
		self.set_fps(float(1.0/(self.exposure_time/1000)))
			
	def set_fps(self,frame_rate):
		time_mult = int(1./float(frame_rate)*float(1000)/0.05)
		time_mult = self.int2hexlist(time_mult)
		self.dev.write(0x01,[0x64,0x02,time_mult[0],time_mult[1]])
		self.fps = frame_rate
	
	def set_main_clock_freq(self):
		self.dev.write(0x01,[0x32,0x01,0x01])
		time.sleep(0.2)
		
	def set_sensor_Hblanking(self):
		self.dev.write(0x01,[0x36,0x01,0x02])
		
	def get_frame(self):
		img_len = self.res[0]*self.res[1]/2
			
		while True:
			array1=[]
			array2=[]

			# Get Camera Trigger State - Checks to see if settings match those set
			self.dev.write(0x01, [0x35, 0x01, 0x00])
			triggerS = self.dev.read(0x81, 0x08)
        
			# Get Image Data - Asks for data to be prepared
			self.dev.write(0x01, [0x34, 0x01, 0x01])
			# By now data was ready so reading of data is begun
			for x in range(0, int(img_len/512+1)):                                 #debugging: xrange in Python3 is range
				array1.extend(self.dev.read(0x82,0x200))
				array2.extend(self.dev.read(0x86,0x200))
			
			# Get Current Frame Property
			self.dev.write(0x01, [0x33, 0x01, 0x00])
			frameP = self.dev.read(0x81, 0x18)
			
			if frameP[16] == 0:
				break
			
		SSAT = []

		for x in range(0,int(self.res[1]/2)):
			arrayStart = x*self.res[0]
			arrayEnd = arrayStart+self.res[0]
			SSAT.extend(array1[arrayStart:arrayEnd])
			SSAT.extend(array2[arrayStart:arrayEnd])	
	
		image = np.reshape(SSAT,[self.res[1],self.res[0]])
		return image
			
	def saveimage(self,directory,imagetag,framenum,image):
		filename = directory + os.sep + imagetag +"_"+ str(framenum) + '.png' 
		f = open(filename,'wb')
		w = png.Writer(self.res[0],self.res[1],greyscale=True,bitdepth=8)
		w.write(f,image)
		if(sys.platform!='win32'):
			setpermissions(filename)
		f.close()

        
	def write(self,command,parameters):
		command_list = [command,len(parameters)]
		for i in parameters:
			command_list.append(i)
		return self.dev.write(0x01,command_list)

	def read(self,length,endpoint=0x81):
		return self.dev.read(endpoint,length)
			
	def int2hexlist(self,num):
		lsb = num & 0xFF
		msb = (num & 0xFF00) >> 0x8
		return [msb,lsb]
			
	def hexlist2int(self,hexlist):
		return (hexlist[0] << 0x8 + hexlist[1])
		
	def analyzeframe(self,image):
		imavg = np.average(image)
		immax = np.amax(image)
		print('Image average = ' + str(imavg) + ' ' + 'Image maximum = ' + str(immax))
		if immax == 255:
			saturation_flag = -1
			print('Image saturated!')
		else:
			saturation_flag = 0
		#return imavg, saturation_flag
		
	def getandsaveframe(self,directory):
		currenttime = time.time()-starttime
		
#		imagetag = 'Camera'   # this is where the name of the image is set - we can change this in the acquisition loop
		
		img = camera.get_frame()
		camera.saveimage(directory,i,imagetag,img)
		print("Got frame #" + str(i+1))
		self.analyzeframe(img)

		
def createworkingdir(foldername):
	currentdir = os.getcwd()
	directory = currentdir + os.sep + foldername
	
	if not os.path.exists(directory):
		os.mkdir(directory)
	if(sys.platform!='win32'):
		setpermissions(directory)
	return directory

def setpermissions(path):
    uid = pwd.getpwnam("joshbrake").pw_uid
    gid = grp.getgrnam("user").gr_gid

    os.chmod(path,0o755)
    os.chown(path,uid,gid)

def getuserinput():
	print("FMightex Camera")
	#debugging: raw_input() in python3 is input()                         
	print("Type new folder name for the data. WARNING: If you do not change folder name, files will be overwritten!")
	foldername = input()
	
	print("How many frames?")
	numframes = input()
	print("----------------------")
	return numframes,foldername

camera=Camera()

print("Ready!")

# Home the Stage
If you're having trouble with timeout, adjust the device settings in the start-up section.

In [ ]:
# Home the device and check the result.
reply = device.home()
if check_command_succeeded(reply):
    print("Device Homed.")
else:
    print("Device home failed.")
    exit(1)
    

reply= device2.home()
if check_command_succeeded(reply):
    print("Device 2 Homed.")
else:
    print("Device 2 home failed.")
    exit(1)
    
print("Set!")

# Get Data!
This is the code to run the actual movement and acquisition

In [20]:
# note - the sparse map needs to be a matrix of size nx2
# this code just moves to each point as they are arranged in the matrix
# we could probably write a few lines to convert a matrix into a raster order
# but for now, that ordering should be done before importing here

# Import Sparse Map

# #import the sparse map as an array
# fname = "sparsemap.txt" # name of the file
# # Import Sparsemap & Convert sparse map to mirosteps 
# #     Conversion: Zaber T-LSM100a linear stage has microstep size of 0.047625um
# #                 Sparsemap is created in microns
# #                 1 microstep = input[microns]/.047625[microns/microstep]
# sparse=np.loadtxt(fname)/.047625
# sparse=sparse.astype(int)

#sparse=np.random.randint(20000, 30000, size= (3,2))  #a is the matrix randomly created between 13000 ans 36000, 10 pairs
#sparse=sparse*4;
# print (sparse)

# Set start time to time stamp frames
starttime = time.time()

# move to each of the points in the sparse map 

# for index in range(len(sparse)):
# #    print ("Moving x to:", sparse[index,0])
#     print ("Moving x to position",index+1)
#     if not safe_move(device, sparse[index,0]): #moving the x axis (device) randomly using position ofrandom matrix column 0
#         print("Failed to move device 1.")
#         exit(1)
    
# #    print ("Moving y to:", sparse[index,1])               #moving the y axis (device2) randomly using position of random matrix (column 1)
#     print ("Moving y to position",index+1)
#     if not safe_move(device2, sparse[index,1]):
#         print("Failed to move device 2.")
#         exit(1)

# move the full matrix with 1mm step size

# Ask the user how many frames per point and what folder to save files in
[numframes,foldername] = getuserinput()
# create a new folder and change directory to that folder
createworkingdir(foldername)


for yindex in range(100):
    print ("Moving y to position",yindex+1)
    y = (yindex+1)*1000/0.047625;
    y = int(y)
    if not safe_move(device2,y):
        print("Failed to move device 2.")
        exit(1)
        #time.sleep(3)
    for xindex in range(100):
        print ("Moving x to position",xindex+1)
        x = (xindex+1)*1000/.047625;
        x = int(x)
        if not safe_move(device,x):
            print("Failed to move device 1.")
            exit(1)
            #time.sleep(2)

# Camera Acquisiton
        if __name__ == "__main__":

            imagetag = 'Camera_x'+str(xindex+1)+'_y'+str(yindex+1) #this becomes your filename
            
            if numframes == 'go':
                i = 0
                while True:
                    camera.getandsaveframe(directory)
                    i +=1
		
            else:
                for i in range(0,int(numframes)):
                    camera.getandsaveframe(directory)    
        #time.sleep(2) 


#port.close()

NameError: name 'time' is not defined